# Collision Avoidance - Live Demo(AlexNetデモ)

このnotebookでは、JetBotが「free（直進する）」か「blocked（旋回する）」かを検出する学習済みモデルを使用して、JetBotの衝突回避動作を確認できます。

## Load the trained model(学習済みモデルの読み込み)

``train_model.ipynb``ノートブックの指示に従って、すでに``best_model.pth``が存在している事を前提とします。

Pytorch modelの初期化を下記コードでおこないます。

In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

``train_model.ipynb``で学習した``best_model.pth``から学習の重みづけをロードします。

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

デフォルトではモデルのweightはCPUで処理されるため、GPUを利用するようにモデルを設定します。

In [ ]:
print("before: {}".format(model.classifier[6].weight.type()))
device = torch.device('cuda')
model = model.to(device)
print("after: {}".format(model.classifier[6].weight.type()))

モデルを読み込みましたが、まだ少し問題があります。  
学習時の入力画像フォーマットと、OpenCVのカメラ画像フォーマットは一致しません。  
これを解消するために、 いくつかの前処理を行う必要があります。これらは、下記の手順になります。

1. cudnnはHWC(Height x Width x Channel)をサポートしません。そのため画像(HWC layout)からTensor(CHW layout)に変換します。
2. トレーニング中に使ったのと同じパラメーターを使用して正規化します。カメラから取得した画像データにはRGBの値があり、1ピクセルはRGBをそれぞれint型で[0、255]の範囲で表したものになります。モデルの入力に使う画像データのRGBはfloat型で[0.0、1.0]の範囲になるため、カメラ画像のRGBをそれぞれ255で割る必要があります。
3. カメラ画像をGPUメモリに転送します。
4. 入力画像データをバッチ配列に変更します。学習時に複数の画像を入力に持っているため、予測時にも1枚の画像であっても入力データは配列にする必要があります。

In [ ]:
import cv2
import numpy as np

# この値はpytorch ImageNetの学習に使われた正規化のパラメータです。
# カメラ画像はこの値で正規化することが望ましいでしょう。
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    x = camera_value
    # 学習時の画像データはtorchvision.datasets.ImageFolderを使って読み込んでいるため、モデルはRGBフォーマットの画像で学習しています。
    # カメラ映像はOpenCVで読み込んでいるため、OpenCV標準のBGRフォーマットをRGBフォーマットに変換します。
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    # 画像をTensor形式に変換します。つまりHWCをCHWに変換します。
    x = x.transpose((2, 0, 1))
    # float32に変換します。
    x = torch.from_numpy(x).float()
    # 正規化します。
    x = normalize(x)
    # GPUデバイスを利用します。
    x = x.to(device)
    # バッチ配列に変換します。
    x = x[None, ...]
    return x

すばらしい、これでカメラ画像をニューラルネットワークの入力フォーマットに変換するための、pre-processing関数を定義できました。　

次はカメラを使うので、一度カメラ用のデーモンを再起動しておきます。

In [ ]:
!echo jetbot | sudo -S systemctl restart nvargus-daemon

カメラを起動し、画面に表示します。また、`blocked`である確率を表示するスライダーを作成します。

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=10)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

モーターを制御するためにrobotインスタンスを生成します。

In [ ]:
from jetbot import Robot

robot = Robot()

次は、カメラの画像が更新されるたびに呼び出される関数を生成します。この関数は、下記ステップを実行します。

1. カメラ画像をPre-processingにかけてモデル入力データに変換する
2. モデル推論の実行
3. 推論結果が50%以上の確率で`blocked`の場合は、左に曲がります。それ以外の場合は前進します。

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(0.6)
    else:
        robot.left(0.6)
    
    time.sleep(0.001)

モデル推論からJetBotの動作までを実行する関数を作成しました。  
今度はそれをカメラと連動して処理する必要があります。

JetBotでは、traitletsライブラリのSingletonConfigurableを継承した[Camera](https://github.com/NVIDIA-AI-IOT/jetbot/blob/master/jetbot/camera.py)クラスを実装することで実現しています。

## JetBotを動かしてみよう
次のコードで``start jetbot``ボタンと``stop jetbot``ボタンを作成します。  
``start jetbot``ボタンを押すとモデルの初期化が実行され、JetBotが動作し始めます。  
``stop jetbot``ボタンを押すとJetBotが停止します。  
最初の1フレームの実行時にメモリの初期化が実行されるので、ディープラーニングではどんなモデルも最初の1フレームの処理はすこし時間がかかります。

In [ ]:
import ipywidgets
import time

model_start_button = ipywidgets.Button(description='start jetbot')
model_stop_button = ipywidgets.Button(description='stop jetbot')

def start_model(c):
    update({'new': camera.value})  
    camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera
model_start_button.on_click(start_model)
    
def stop_model(c):
    camera.unobserve(update, names='value')
    time.sleep(1)
    robot.stop()
model_stop_button.on_click(stop_model)

model_widget = ipywidgets.VBox([
    ipywidgets.HBox([image, blocked_slider]),
    ipywidgets.HBox([model_start_button, model_stop_button])
])

display(model_widget)

やったね！JetBotが動作している場合、新しいカメラフレームごとに画像変換、モデル推論、JetBot制御処理が実行されています。

あなたがデータセットを作ったコース上にJetBotを置いてください。そして障害物に到達したときの動作を確認する事ができるでしょう。

### Conclusion

以上がライブデモです。今はJetBotが衝突を回避しながら走行しているのではないでしょうか？　

collision avoidanceが上手く行かない場合、正しく走行できるように失敗しやすい場所でさらにデータを追加してください。  
このようにうまくいかない場所を中心にデータを収集すれば、JetBotはさらによく動作するはずです。

### おまけ

ResNet18のサンプルもあります。時間がまあった人は、``train_model_resnet18.ipynb``を実行し、resnet18モデルで学習してください。  ResNet18モデルは、続けてTensorRTモデルに変換して使うことができます。

[train_model_resnet18.ipynb](./resnet18/train_model_resnet18.ipynb) をクリックし、移動します。